In [1]:
from eoxhub import check_compatibility
check_compatibility("user-2022.10-14", dependencies=[])

# Crop-classification using Sentinel-2 time-series

This notebook implements a crop classification algorithm for Sentinel-2 time-series based on deep learning. The input time-series are derived from the signals computed by the Eurocrops BYOA. The method here implemented is described in more detail in the [crop-classification marker blog-post](https://medium.com/sentinel-hub/area-monitoring-crop-type-marker-1e70f672bf44).

For more examples on how to create markers for monitoring agricultural activity using Sentinel-2 signals, consult [this blog series](https://medium.com/sentinel-hub/area-monitoring-concept-effc2c262583).

This notebook will use a sample of pre-downloaded signals, and can be run on a CPU-based instance or laptop.

**Table of Contents**:

 0. [Constants](#constants)
 1. [Retrieve signals and labels](#retrieve-signals)
 2. [AI-ready dataset](#dataset)
 3. [Model training](#model-training)
 4. [Model evaluation](#model-evaluation)

In [2]:
import os
import sys

!{sys.executable} -m pip install torch torchvision
os.environ['EDC_PATH'] + "/notebooks/contributions/eurocrops-model"
sys.path.append(os.path.join(os.environ['EDC_PATH'] + "/notebooks/contributions/eurocrops-model"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/776.4 MB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/776.4 MB 165.4 MB/s eta 0:00:05

     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/776.4 MB 183.4 MB/s eta 0:00:05

     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/776.4 MB 215.7 MB/s eta 0:00:04

     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/776.4 MB 251.0 MB/s eta 0:00:03

...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 19.1/19.1 MB 28.6 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 19.1/19.1 MB 28.6 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 19.1/19.1 MB 28.6 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 19.1/19.1 MB 28.6 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 15.5 MB/s eta 0:00:00


In [3]:
import json
import logging
import sys
import zipfile

import geopandas as gpd
import numpy as np
import pandas as pd
import subprocess
from lstm import LSTM
from polygon import PolyDataset
from transforms import get_sample_n_timestamps
from utils import test, train
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import DataLoader

from sentinelhub import parse_time

## 0. Constants <a name=constants></a>

This section initialises paths, constants and utility functions used in the notebook.

In [4]:
# path to local folders, change as desired
INPUT_FOLDER = "./input"
RESULTS_FOLDER = "./output"

# name of files and folders of downloaded signals, do not change
DATAFILE = "ml-example-data.zip"
DOWNLOAD_URL = f"https://sinergise0-my.sharepoint.com/:u:/g/personal/nejc_vesel_sinergise_com/ETMx7NG-JHpBntNMJfnsCOMBVuEegDjYq8WtTmJYl8tZ-A?e=Ck5opE&download=1"
EUROCROPS_GPKG = "input_geometries.gpkg"
SIGNALS_FOLDER = "ml-example-signals"


# utility function to read json payload into a dataframe
def stats_to_df(stats_data):
    """Transform Statistical API response into a pandas.DataFrame"""
    df_data = []

    for single_data in stats_data["data"]:
        df_entry = {}
        is_valid_entry = True

        df_entry["interval_from"] = parse_time(single_data["interval"]["from"]).date()
        df_entry["interval_to"] = parse_time(single_data["interval"]["to"]).date()

        for output_name, output_data in single_data["outputs"].items():
            for band_name, band_values in output_data["bands"].items():
                band_stats = band_values["stats"]
                if band_stats["sampleCount"] == band_stats["noDataCount"]:
                    is_valid_entry = False
                    break

                for stat_name, value in band_stats.items():
                    col_name = f"{output_name}_{band_name}_{stat_name}"
                    if stat_name == "percentiles":
                        for perc, perc_val in value.items():
                            perc_col_name = f"{col_name}_{perc}"
                            df_entry[perc_col_name] = perc_val
                    else:
                        df_entry[col_name] = value

        if is_valid_entry:
            df_data.append(df_entry)

    return pd.DataFrame(df_data)


# utility function to log training progress
def define_logger(logger_name) -> logging.Logger:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter("[%(asctime)s] {%(filename)s:%(lineno)d} %(levelname)s - %(message)s")

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(formatter)
    logger.addHandler(stdout_handler)
    return logger

## 1. Retrieve signals and labels <a name=retrieve-signals></a>

In this Section, signals are read from JSON files, while the labels used during model training are extracted from the Eurocrops dataset. For this notebook, sample files are provided and downloaded locally in the following cells. 

In [5]:
if not os.path.exists(INPUT_FOLDER):
    os.mkdir(INPUT_FOLDER)

In [6]:
wget_str = f"wget {DOWNLOAD_URL} -O {os.path.join(INPUT_FOLDER, DATAFILE)}"
subprocess.call(wget_str.split(" "))

--2022-10-24 13:37:03--  https://sinergise0-my.sharepoint.com/:u:/g/personal/nejc_vesel_sinergise_com/ETMx7NG-JHpBntNMJfnsCOMBVuEegDjYq8WtTmJYl8tZ-A?e=Ck5opE&download=1
Resolving sinergise0-my.sharepoint.com (sinergise0-my.sharepoint.com)... 13.107.136.9, 13.107.138.9
Connecting to sinergise0-my.sharepoint.com (sinergise0-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: /personal/nejc_vesel_sinergise_com/Documents/ml-example-data.zip?ga=1 [following]
--2022-10-24 13:37:03--  https://sinergise0-my.sharepoint.com/personal/nejc_vesel_sinergise_com/Documents/ml-example-data.zip?ga=1
Reusing existing connection to sinergise0-my.sharepoint.com:443.
HTTP request sent, awaiting response... 

200 OK
Length: 88085785 (84M) [application/x-zip-compressed]
Saving to: ‘./input/ml-example-data.zip’

     0K .......... .......... .......... .......... ..........  0% 5.54M 15s
    50K .......... .......... .......... .......... ..........  0% 5.40M 15s
   100K .......... .......... .......... .......... ..........  0% 5.93M 15s
   150K .......... .......... .......... .......... ..........  0% 79.5M 11s
   200K .......... .......... .......... .......... ..........  0% 5.98M 12s
   250K .......... .......... .......... .......... ..........  0% 55.4M 10s
   300K .......... .......... .......... .......... ..........  0%  113M 9s
   350K .......... .......... .......... .......... ..........  0% 73.8M 8s
   400K .......... .......... .......... .......... ..........  0% 6.52M 8s
   450K .......... .......... .......... .......... ..........  0% 69.1M 8s
   500K .......... .......... .......... .......... ..........  0% 62.4M 7s
   550K .......... .......... .......... .......... ...

 .......... ..........  9% 10.8M 2s
  8300K .......... .......... .......... .......... ..........  9%  107M 2s
  8350K .......... .......... .......... .......... ..........  9% 45.1M 2s
  8400K .......... .......... .......... .......... ..........  9%  310M 2s
  8450K .......... .......... .......... .......... ..........  9%  373M 2s
  8500K .......... .......... .......... .......... ..........  9%  376M 2s
  8550K .......... .......... .......... .......... ..........  9%  371M 2s
  8600K .......... .......... .......... .......... .......... 10%  255M 2s
  8650K .......... .......... .......... .......... .......... 10%  267M 2s
  8700K .......... .......... .......... .......... .......... 10%  213M 2s
  8750K .......... .......... .......... .......... .......... 10%  334M 2s
  8800K .......... .......... .......... .......... .......... 10%  110M 2s
  8850K .......... .......... .......... .......... .......... 10%  325M 2s
  8900K .......... .......... .......... .......... 


 24200K .......... .......... .......... .......... .......... 28%  532K 1s
 24250K .......... .......... .......... .......... .......... 28%  239M 1s
 24300K .......... .......... .......... .......... .......... 28%  211M 1s
 24350K .......... .......... .......... .......... .......... 28%  268M 1s
 24400K .......... .......... .......... .......... .......... 28%  106M 1s
 24450K .......... .......... .......... .......... .......... 28%  204M 1s
 24500K .......... .......... .......... .......... .......... 28%  366M 1s
 24550K .......... .......... .......... .......... .......... 28%  271M 1s
 24600K .......... .......... .......... .......... .......... 28% 72.7M 1s
 24650K .......... .......... .......... .......... .......... 28% 78.1M 1s
 24700K .......... .......... .......... .......... .......... 28%  333M 1s
 24750K .......... .......... .......... .......... .......... 28%  339M 1s
 24800K .......... .......... .......... .......... .......... 28%  285M 1s
 24850K ...

........ 56% 1.40M 0s
 48400K .......... .......... .......... .......... .......... 56% 55.8M 0s
 48450K .......... .......... .......... .......... .......... 56% 59.1M 0s
 48500K .......... .......... .......... .......... .......... 56%  311M 0s
 48550K .......... .......... .......... .......... .......... 56%  266M 0s
 48600K .......... .......... .......... .......... .......... 56%  118M 0s
 48650K .......... .......... .......... .......... .......... 56% 99.1M 0s
 48700K .......... .......... .......... .......... .......... 56% 52.2M 0s
 48750K .......... .......... .......... .......... .......... 56%  344M 0s
 48800K .......... .......... .......... .......... .......... 56%  301M 0s
 48850K .......... .......... .......... .......... .......... 56%  362M 0s
 48900K .......... .......... .......... .......... .......... 56%  370M 0s
 48950K .......... .......... .......... .......... .......... 56%  352M 0s
 49000K .......... .......... .......... .......... .......... 57%

 .......... .......... 85%  133K 0s
 73500K .......... .......... .......... .......... .......... 85%  245M 0s
 73550K .......... .......... .......... .......... .......... 85%  299M 0s
 73600K .......... .......... .......... .......... .......... 85%  245M 0s
 73650K .......... .......... .......... .......... .......... 85%  360M 0s
 73700K .......... .......... .......... .......... .......... 85%  364M 0s
 73750K .......... .......... .......... .......... .......... 85%  363M 0s
 73800K .......... .......... .......... .......... .......... 85%  259M 0s
 73850K .......... .......... .......... .......... .......... 85%  333M 0s
 73900K .......... .......... .......... .......... .......... 85%  354M 0s
 73950K .......... .......... .......... .......... .......... 86%  357M 0s
 74000K .......... .......... .......... .......... .......... 86%  322M 0s
 74050K .......... .......... .......... .......... .......... 86%  360M 0s
 74100K .......... .......... .......... .......... 

0

In [7]:
# unzip files
with zipfile.ZipFile(os.path.join(INPUT_FOLDER, DATAFILE), "r") as zip_ref:
    zip_ref.extractall(INPUT_FOLDER)

In [8]:
!ls {INPUT_FOLDER}

input_geometries.gpkg  ml-example-data.zip  ml-example-signals


Read signals from JSON files as returned by Statistical API and create a dataframe.

In [9]:
dfs = []
for result_json in os.listdir(os.path.join(INPUT_FOLDER, "ml-example-signals")):
    with open(os.path.join(INPUT_FOLDER, "ml-example-signals", result_json)) as f:
        result = json.load(f)

    result_df = stats_to_df(result["response"])
    result_df["identifier"] = int(result["identifier"])
    dfs.append(result_df)

signals = pd.concat(dfs)

Check size of dataset and one time observation.

In [10]:
len(signals)

157500

In [11]:
signals.iloc[0]

interval_from            2021-01-05
interval_to              2021-01-06
clm_B0_min                      1.0
clm_B0_max                      1.0
clm_B0_mean                     1.0
                            ...    
bands_B12_mean             0.028348
bands_B12_stDev            0.003333
bands_B12_sampleCount            96
bands_B12_noDataCount            65
identifier                   331393
Name: 0, Length: 93, dtype: object

Read labels from provided geopackage file. These labels can be directly retrieved from GeoDB.

In [12]:
eurocrops_gdf = gpd.read_file(os.path.join(INPUT_FOLDER, EUROCROPS_GPKG))

Merge signals and crop labels into a single dataframe, by looking at the identifier of the field of interest (FOI).

In [13]:
eurocrops_signals = pd.merge(eurocrops_gdf, signals, on="identifier")

In [14]:
eurocrops_signals.iloc[0]

identifier                                         296351
created_at               2022-07-06T21:34:12.027081+00:00
modified_at                                          None
_fid                                                  NaN
fs_kennung                                            NaN
                                       ...               
bands_B12_max                                      0.3369
bands_B12_mean                                    0.28427
bands_B12_stDev                                   0.03697
bands_B12_sampleCount                                 112
bands_B12_noDataCount                                  72
Name: 0, Length: 121, dtype: object

## 2. AI-ready dataset <a name=dataset></a>

This Section adds some features to be added to the raw bands which will be used by the model.

In [15]:
# add column for cloud probability
eurocrops_signals["CLP"] = eurocrops_signals["clp_B0_mean"] / 255
# compute NDVI
eurocrops_signals["NDVI"] = (eurocrops_signals["bands_B7_mean"] - eurocrops_signals["bands_B3_mean"]) / (
    eurocrops_signals["bands_B7_mean"] + eurocrops_signals["bands_B3_mean"]
)
# compute day-of-year from timestamp
eurocrops_signals["DOY"] = eurocrops_signals.interval_from.apply(lambda x: x.timetuple().tm_yday)

In [16]:
# get name of columns to be used as features, i.e. mean values of raw bands
feature_cols = [x for x in eurocrops_signals.columns if x.startswith("bands_") and x.endswith("_mean")]
# name of utility features
doy_feature = "DOY"
crop_type_feature = "ec_hcat_c"
crop_name_feature = "ec_hcat_n"
label_feature = "label"
identifier_feature = "identifier"

Map all the possible crop-types to specific groups assigned in Eurocrops.

In [17]:
crop_id_to_label_mapping = {val: idx for idx, val in enumerate(eurocrops_signals[crop_type_feature].unique())}

In [18]:
crop_id_to_name_mapping = {
    crop_id: crop_name
    for crop_id, crop_name in eurocrops_signals[[crop_type_feature, crop_name_feature]].drop_duplicates().values
}

In [19]:
eurocrops_signals[label_feature] = eurocrops_signals[crop_type_feature].map(crop_id_to_label_mapping)

Split the signals into a training and validation set. This datasets are demonstrative only, as in reality, a larger dataset would be required, and more robust validation strategies required to robustly estimate the performance of the model.

In [20]:
train_ids, val_ids = train_test_split(
    eurocrops_signals[identifier_feature].unique(), train_size=0.6, test_size=0.4, random_state=42
)

In [21]:
train_df = eurocrops_signals[eurocrops_signals[identifier_feature].isin(train_ids)]
val_df = eurocrops_signals[eurocrops_signals[identifier_feature].isin(val_ids)]

Create the training and validation datasets to be used for model training and validation.

In [22]:
train_poly_dataset = PolyDataset(
    train_df,
    feature_cols=feature_cols,
    label_col=label_feature,
    poly_id_col=identifier_feature,
    doys_col=doy_feature,
    online_transform=get_sample_n_timestamps(40),
)

val_poly_dataset = PolyDataset(
    val_df,
    feature_cols=feature_cols,
    label_col=label_feature,
    poly_id_col=identifier_feature,
    doys_col=doy_feature,
    online_transform=get_sample_n_timestamps(40),
)

## 3. Model training <a name=model-training></a>

In this Section, a LSTM model is trained on the signals for estimation of crop-type. The parameters of the LSTM might need tuning to different use-cases.

In [23]:
BATCH_SIZE = 16
N_WORKERS = 4
SHUFFLE = True

In [24]:
train_loader = DataLoader(dataset=train_poly_dataset, batch_size=BATCH_SIZE, num_workers=N_WORKERS, shuffle=SHUFFLE)

val_loader = DataLoader(dataset=val_poly_dataset, batch_size=BATCH_SIZE, num_workers=N_WORKERS, shuffle=False)

Initialise the model.

In [25]:
lstm = LSTM(
    input_dim=len(feature_cols),
    n_classes=eurocrops_signals[label_feature].nunique(),
    hidden_dims=128,
    num_rnn_layers=3,
    dropout=0.2,
    bidirectional=True,
    use_batchnorm=False,
    use_layernorm=True,
)

Initialise the optimiser.

In [26]:
optimizer = Adam(
    filter(lambda x: x.requires_grad, lstm.parameters()),
    betas=(0.9, 0.98),
    eps=1e-09,
    lr=0.001,
)

Initialise the logger.

In [27]:
logger = define_logger("Training")

Train the model !!

In [28]:
lstm = train(lstm, optimizer, train_loader, val_loader, 30, verbose=False, logger=logger)

[2022-10-24 13:38:51,372] {utils.py:87} INFO - Epoch 0: train loss 2.235 | val loss 1.877 | val acc = 0.482


[2022-10-24 13:39:45,773] {utils.py:87} INFO - Epoch 1: train loss 1.783 | val loss 1.665 | val acc = 0.566


[2022-10-24 13:40:38,163] {utils.py:87} INFO - Epoch 2: train loss 1.669 | val loss 1.716 | val acc = 0.543


[2022-10-24 13:41:32,762] {utils.py:87} INFO - Epoch 3: train loss 1.610 | val loss 1.645 | val acc = 0.562


[2022-10-24 13:42:27,675] {utils.py:87} INFO - Epoch 4: train loss 1.502 | val loss 1.709 | val acc = 0.532


[2022-10-24 13:43:20,078] {utils.py:87} INFO - Epoch 5: train loss 1.445 | val loss 1.557 | val acc = 0.584


[2022-10-24 13:44:13,485] {utils.py:87} INFO - Epoch 6: train loss 1.394 | val loss 1.441 | val acc = 0.623


[2022-10-24 13:45:07,382] {utils.py:87} INFO - Epoch 7: train loss 1.371 | val loss 1.532 | val acc = 0.596


[2022-10-24 13:46:00,682] {utils.py:87} INFO - Epoch 8: train loss 1.349 | val loss 1.423 | val acc = 0.625


## 4. Model evaluation <a name=model-evaluation></a>

This Section evaluates the performance of the trained model on the validation dataset. Perfomance is displayed as a confusion matrix, where estimated and reference crop-types are compared.

In [ ]:
import numpy as np

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / np.sum(cm).astype('float')
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = np.nanmax(cm) / 1.5 if normalize else np.nanmax(cm) / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.2f}; misclass={:0.2f}'.format(accuracy, misclass))
    plt.show()


In [ ]:
predictions, targets, polygon_ids, logprobabilities = test(lstm, val_loader)

In [ ]:
cm = confusion_matrix(targets, predictions, labels=list(crop_id_to_label_mapping.values()))

In [ ]:
plot_confusion_matrix(cm, target_names=[crop_id_to_name_mapping[x] for x in crop_id_to_label_mapping.keys()])